# Parse Input Midi Files

Analyse Midi files to get the features I am looking for and get a few bars of melody

In [2]:
from music21 import *
from pprint import pprint
import glob

In [ ]:
import music21 as music21

In [ ]:
!sudo pip install --upgrade music21

In [ ]:
us = environment.UserSettings()
for key in sorted(us.keys()):
    if key != "localCorpusPath":
        print("Key : ", str(key), "\nValue: ", str(us[key]), "\n")

us["midiPath"] = "/snap/bin/musescore.mscore"
us["musicxmlPath"] = "/snap/bin/musescore.mscore"
us["musescoreDirectPNGPath"] = "/snap/bin/musescore.mscore"
us["lilypondPath"] = "/home/fireredninja/bin/lilypond"

In [ ]:
!ls ./data

In [ ]:
!rm ./output/*.mid

In [40]:
# Look at all the functions and attributes
def printAttributesAndFunctions(object):
#     x = True
#     for part in object:
#         if x:
#     #         pprint(dir(note))
#             x = False
    for func in dir(object):
        try:
            f = getattr(object, func)
            print("---------------PASS---------------")
            if callable(f): # is it a function
                # call it
                print(f"PRINTING CALLABLE FUNCTION ----- {func}")
                print(eval(f'note.{func}()'), "\n\n\n")
                pass
            else:
                # just print the attribute
                print(f"PRINTING ATTRIBUTE ------ {func}")
                print(eval(f'note.{func}'), "\n\n\n")
                pass
        except:
            try:
                print(eval(f'note.{func}'), "\n\n\n")
                pass
            except:
                print("---------------FAIL---------------")
                print(f'{func}', "\n\n\n")
                pass

In [ ]:
# part0 = midi.parts[0]

# for bar in part0.measures(0, None):
#     print("--------------")
#     try:
#         for el in bar.flat:
#             print(el)
#             if isinstance(el, note.GeneralNote):
#                 print("aaaaaa")
#                 print(el.quarterLength, el.pitch)
#     except:
#         print("failed")
#         pass

In [ ]:
def transpose(s):
    k = s.analyze('key')
    print(f"Transposing from {k} to C-major")
    i = interval.Interval(k.tonic, pitch.Pitch('C'))
    sNew = s.transpose(i)
    # do something with sNew
    return sNew

In [ ]:
# Checks if the measure meets the requirements
def metRequirements(measure):
    pprint("Checking if requirements are met")
    requirementsMet = False
                    
    # Loop through all the notes
    for element in measure.flat:
        if isinstance(element, note.Note):
            requirementsMet = True
        elif isinstance(element, chord.Chord):
            requirementsMet = False
            print("Chord found")
            print("Requirements not met")
            break
            # requirementsMet = metRequirements(element)
    
    return requirementsMet

In [ ]:
def getTimeSignature(measure):
    if "_elements" in measure.__dict__:
        for element in measure.__dict__["_elements"]:
            if isinstance(element, meter.TimeSignature):
                return element
    return None

In [ ]:
def writeToMidi(file, part, partCounter, noOfMeasures):
    try:
        print(f"generating midi file {file[:-4]}-Part{partCounter}.mid")
        output = stream.Score(id='mainScore')
        filePart = stream.Part(id='part0')
        filePart.append([part.measures(0,noOfMeasures)])
        output.insert(0, filePart)
        # output = transpose(output)
        filename = file[file.rfind("/"):-3].replace(" ", "") + ".mid"
        output.write(fmt="midi", fp=f"./output/{filename}.mid")
    except:
        pass

In [ ]:
def createDataset(location, noOfMeasures):

    # loop through all the midi files in the location
    for file in glob.glob(f"{location}*.mid"):
        
        print("----------------NEW FILE----------------")

        midi = converter.parse(file)
        print(f"Parsing {file}")

        
        # Loop through all the parts in the score
        # each part is the score for an instrument
        partCounter = 0
        print("No. of parts ", len(midi.parts))
        for part in midi.parts:
            
            print("----------------NEW PART----------------")
            
            try:
                # Loop through all the measures
                print("Key: ", part.analyze('key'))
                print("Currently on part ", partCounter)
                print("No. of measures ", len(part.measures(0, None)))
                partCounter+=1 # TODO: move this to end of for loop
            except:
                continue
            
            if (len(part.measures(0, None)) < noOfMeasures):
                continue


            # assume wrong time signature and wrong clef
            correctTimeSignature = False
            correctClef = False
            

            # keeps track of which measure we are on
            measureCounter = 0
            
            # checks if the measures meet the requirements
            requirementsMet = False
            
            # loop over all of the measures in the score
            for measure in part.measures(0, noOfMeasures):
                print("----------------------------------------")
                print("Measure number: ", measureCounter)
                measureCounter+=1
            
#                 print("-----MEASURE--------")
#                 pprint(measure.__dict__)
#                 print("--------------------")
                
                # check if the score/measure uses the correct clef
                if hasattr(measure, 'clef'):
                    if hasattr(measure.clef, 'line'):
                        if measure.clef.line == 2:
                            print("Clef: TrebleClef")
                            correctClef = True
                        else:
                            print("Wrong Clef Found")
                            correctClef = False
                            break
                
                # check if the score/measure uses the correct time signature
                timeSignature = getTimeSignature(measure)
                if timeSignature != None:
                    if timeSignature.ratioString == '4/4':
                        print("Time Signature: 4/4")
                        correctTimeSignature = True
                    else:
                        print("Wrong Time Signature Found")
                        correctTimeSignature = False
                        break
                else:
                    print("No attribute for TimeSignature")

                # skip measure  if
                if not correctClef or not correctTimeSignature:
                    print("Incorrect Clef or TimeSignature")
                    continue

                requirementsMet = metRequirements(measure)
                if requirementsMet:
                    pprint("Requirements met")
                    continue
                else:
                    pprint("Requirements not met")
                    break

            if requirementsMet:
                writeToMidi(file, part, partCounter, noOfMeasures)
    print("Finished")

In [ ]:
createDataset("./data/", 3)

# Midi to Numpy

Create a numpy representation of the midi files for the Deep Learning model

In [16]:
file = "./output/Pokemon-LugiasSong..mid.mid"
midi = converter.parse(file)
print(f"Parsing {file}")

Parsing ./output/Pokemon-LugiasSong..mid.mid


In [17]:
midi.show('text')

{0.0} <music21.stream.Part 0x2123d785940>
    {0.0} <music21.instrument.Flute Flute>
    {0.0} <music21.instrument.Flute Flute>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
    {0.0} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {0.0} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {0.0} <music21.key.Key of G major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Note E>
    {1.0} <music21.note.Note G>
    {2.0} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {2.0} <music21.note.Note F#>
    {2.3333} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {2.6667} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {3.0} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {3.25} <music21.note.Note D>
    {3.3333} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {3.75} <music21.tempo.MetronomeMark andantino Quarter=80.0>
    {3.75} <music21.note.Note E>
    {4.0} <music21.tempo.MetronomeMark andantino Q

In [ ]:
printAttributesAndFunctions(midi)

In [6]:
pprint(midi.__dict__["_elements"][0].__dict__)

{'_activeSite': <weakref at 0x00000212375DF408; to 'Score' at 0x00000212375C4940>,
 '_activeSiteStoredOffset': 0.0,
 '_atSoundingPitch': 'unknown',
 '_cache': {'Duration': <music21.duration.Duration 11.5>,
            'HighestTime': 11.5,
            'elements': [<music21.instrument.Piano Piano>,
                         <music21.instrument.Piano Piano>,
                         <music21.key.Key of C major>,
                         <music21.meter.TimeSignature 4/4>,
                         <music21.note.Rest rest>,
                         <music21.note.Note C>,
                         <music21.note.Note A>,
                         <music21.note.Note C>,
                         <music21.note.Rest rest>,
                         <music21.note.Note C>,
                         <music21.note.Note A>,
                         <music21.note.Note C>,
                         <music21.note.Rest rest>,
                         <music21.note.Note E>,
                         <music21.note.

In [59]:
part = midi.__dict__["_elements"][0].__dict__["_elements"]

listOfNotes = [element for element in part if isinstance(element, note.GeneralNote)]

pprint(listOfNotes)
# Note
# Rest
# SpacerRest
# Unpitched
# NotRest
# GeneralNote

def noteToFloat(note):
    notes = ["C", "C#", "D-", "D", "D#", "E-", "E", "F", "F#", "G-", "G", "G#", "A-", "A", "A#", "B-", "B"]
    
    notesToFloatDict = dict(dict(zip(notes, range(1, 17, 1))))
    
    pass

def getDuration(note):
    pprint(note.duration.quarterLength)
    pass


getDuration(part[7])


# printAttributesAndFunctions(part[7])

# print(part[7])
for element in listOfNotes:
    if (isinstance(element, note.GeneralNote)):
        getDuration(element)
        
# print(isinstance(part[4], note.GeneralNote))

[<music21.note.Note E>,
 <music21.note.Note G>,
 <music21.note.Note F#>,
 <music21.note.Note D>,
 <music21.note.Note E>,
 <music21.note.Note B>,
 <music21.note.Note E>,
 <music21.note.Note G>,
 <music21.note.Note F#>,
 <music21.note.Note B>,
 <music21.note.Note E>,
 <music21.note.Note G>]
1.0
1.0
1.0
1.25
0.5
0.5
3.0
1.0
1.0
1.25
0.5
0.5
0.5


In [ ]:
for file in glob.glob('./output/*.mid'):
    print("--------------------------------------")
    midi = converter.parse(file)
    print(f"Parsing {file}")
    midi.show("text")